In [1]:
import os
import sys
import numpy as np
root = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(root)

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from predify.utils.training import train_pcoders, eval_pcoders

from networks_2022 import BranchedNetwork
from data.CleanSoundsDataset import CleanSoundsDataset

# Parameters

In [2]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')
BATCH_SIZE = 50
NUM_WORKERS = 2
PIN_MEMORY = True
NUM_EPOCHS = 50

lr = 1E-4
engram_dir = '/mnt/smb/locker/issa-locker/users/Erica/'
checkpoints_dir = f'{engram_dir}hcnn/checkpoints/'
tensorboard_dir = f'{engram_dir}hcnn/tensorboard/'
datafile = f'{engram_dir}seed_542_word_clean_random_order.hdf5'

Device: cuda:0


In [3]:
from pbranchednetwork_all import PBranchedNetwork_AllSeparateHP
PNetClass = PBranchedNetwork_AllSeparateHP
pnet_name = 'all'

In [4]:
!nvidia-smi

Sat May 21 10:22:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 455.45.01    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:1E:00.0 Off |                  N/A |
| 27%   25C    P8    19W / 250W |      3MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load network and optimizer

In [5]:
net = BranchedNetwork()
net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))

/share/issa/users/es3773/temp-hallucnn/src/models/layers.py:78: UserWarning: Inconsistent tf pad calculation in ConvLayer.
  warnings.warn('Inconsistent tf pad calculation in ConvLayer.')
/share/issa/users/es3773/temp-hallucnn/src/models/layers.py:173: UserWarning: Inconsistent tf pad calculation: 0, 1
  warnings.warn(f'Inconsistent tf pad calculation: {pad_left}, {pad_right}')


<All keys matched successfully>

In [6]:
pnet = PNetClass(net, build_graph=True)

In [7]:
pnet.eval()

PBranchedNetwork_AllSeparateHP(
  (backbone): BranchedNetwork(
    (speech_branch): Sequential(
      (conv1): ConvLayer(
        (block): Sequential(
          (0): Conv2d(1, 96, kernel_size=(6, 14), stride=(3, 3), padding=(2, 6))
          (1): ReLU()
        )
      )
      (rnorm1): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool1): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv2): ConvLayer(
        (block): Sequential(
          (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(2, 2), padding=(1, 2))
          (1): ReLU()
        )
      )
      (rnorm2): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool2): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv3): ConvLayer(
        (block): Sequential(
          (0): Conv2d(256, 512, kernel_size=

In [8]:
pnet.to(DEVICE)
optimizer = torch.optim.Adam(
    [{'params':getattr(pnet,f"pcoder{x+1}").pmodule.parameters(), 'lr':lr} for x in range(pnet.number_of_pcoders)],
    weight_decay=5e-4)

# Set up dataset

In [9]:
full_dataset = CleanSoundsDataset(datafile)
n_train = int(len(full_dataset)*0.9)
train_dataset = Subset(full_dataset, np.arange(n_train))
eval_dataset = Subset(full_dataset, np.arange(n_train, len(full_dataset)))
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )
eval_loader = DataLoader(
    eval_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )

# Set up checkpoints and tensorboards

In [10]:
checkpoint_path = os.path.join(checkpoints_dir, f"{pnet_name}")
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
checkpoint_path = os.path.join(checkpoint_path, pnet_name + '-{epoch}-{type}.pth')

# summarywriter
from torch.utils.tensorboard import SummaryWriter
tensorboard_path = os.path.join(tensorboard_dir, f"{pnet_name}")
if not os.path.exists(tensorboard_path):
    os.makedirs(tensorboard_path)
sumwriter = SummaryWriter(tensorboard_path, filename_suffix=f'')

# Train

In [ ]:
loss_function = torch.nn.MSELoss()
for epoch in range(1, NUM_EPOCHS+1):
    train_pcoders(pnet, optimizer, loss_function, epoch, train_loader, DEVICE, sumwriter)
    eval_pcoders(pnet, loss_function, epoch, eval_loader, DEVICE, sumwriter)

    # save checkpoints every 5 epochs
    if epoch % 5 == 0:
        torch.save(pnet.state_dict(), checkpoint_path.format(epoch=epoch, type='regular'))

/home/es3773/.local/lib/python3.7/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Training Epoch: 1 [50/36450]	Loss: 149598.8281
Training Epoch: 1 [100/36450]	Loss: 107713.1719
Training Epoch: 1 [150/36450]	Loss: 87476.9062
Training Epoch: 1 [200/36450]	Loss: 81291.9531
Training Epoch: 1 [250/36450]	Loss: 73679.8906
Training Epoch: 1 [300/36450]	Loss: 75263.9219
Training Epoch: 1 [350/36450]	Loss: 80098.4453
Training Epoch: 1 [400/36450]	Loss: 85437.3594
Training Epoch: 1 [450/36450]	Loss: 86389.7188
Training Epoch: 1 [500/36450]	Loss: 75190.0469
Training Epoch: 1 [550/36450]	Loss: 73881.2344
Training Epoch: 1 [600/36450]	Loss: 70281.7422
Training Epoch: 1 [650/36450]	Loss: 60905.4570
Training Epoch: 1 [700/36450]	Loss: 57642.3086
Training Epoch: 1 [750/36450]	Loss: 57320.9062
Training Epoch: 1 [800/36450]	Loss: 51446.9609
Training Epoch: 1 [850/36450]	Loss: 55782.9570
Training Epoch: 1 [900/36450]	Loss: 55659.3281
Training Epoch: 1 [950/36450]	Loss: 50686.7461
Training Epoch: 1 [1000/36450]	Loss: 58447.4766
Training Epoch: 1 [1050/36450]	Loss: 53968.1836
Training E

Training Epoch: 1 [8650/36450]	Loss: 11932.1279
Training Epoch: 1 [8700/36450]	Loss: 10801.0205
Training Epoch: 1 [8750/36450]	Loss: 10855.8145
Training Epoch: 1 [8800/36450]	Loss: 10741.6475
Training Epoch: 1 [8850/36450]	Loss: 11473.0264
Training Epoch: 1 [8900/36450]	Loss: 11239.7998
Training Epoch: 1 [8950/36450]	Loss: 10894.8906
Training Epoch: 1 [9000/36450]	Loss: 11096.7578
Training Epoch: 1 [9050/36450]	Loss: 10791.1553
Training Epoch: 1 [9100/36450]	Loss: 10535.8066
Training Epoch: 1 [9150/36450]	Loss: 11112.8135
Training Epoch: 1 [9200/36450]	Loss: 11236.3740
Training Epoch: 1 [9250/36450]	Loss: 10811.6787
Training Epoch: 1 [9300/36450]	Loss: 10074.3574
Training Epoch: 1 [9350/36450]	Loss: 10551.3340
Training Epoch: 1 [9400/36450]	Loss: 10886.8379
Training Epoch: 1 [9450/36450]	Loss: 10808.0762
Training Epoch: 1 [9500/36450]	Loss: 10736.1748
Training Epoch: 1 [9550/36450]	Loss: 10720.2217
Training Epoch: 1 [9600/36450]	Loss: 9848.7900
Training Epoch: 1 [9650/36450]	Loss: 1079

Training Epoch: 1 [17200/36450]	Loss: 5748.0859
Training Epoch: 1 [17250/36450]	Loss: 5358.8462
Training Epoch: 1 [17300/36450]	Loss: 5534.0889
Training Epoch: 1 [17350/36450]	Loss: 5459.5054
Training Epoch: 1 [17400/36450]	Loss: 5335.9292
Training Epoch: 1 [17450/36450]	Loss: 5243.0303
Training Epoch: 1 [17500/36450]	Loss: 5255.7012
Training Epoch: 1 [17550/36450]	Loss: 5145.5942
Training Epoch: 1 [17600/36450]	Loss: 5257.8628
Training Epoch: 1 [17650/36450]	Loss: 5171.7632
Training Epoch: 1 [17700/36450]	Loss: 5337.7661
Training Epoch: 1 [17750/36450]	Loss: 5059.0781
Training Epoch: 1 [17800/36450]	Loss: 5040.4346
Training Epoch: 1 [17850/36450]	Loss: 5161.0820
Training Epoch: 1 [17900/36450]	Loss: 5166.5469
Training Epoch: 1 [17950/36450]	Loss: 5367.4844
Training Epoch: 1 [18000/36450]	Loss: 5393.7773
Training Epoch: 1 [18050/36450]	Loss: 5246.8804
Training Epoch: 1 [18100/36450]	Loss: 5282.9321
Training Epoch: 1 [18150/36450]	Loss: 5168.7271
Training Epoch: 1 [18200/36450]	Loss: 46

Training Epoch: 1 [25750/36450]	Loss: 3284.0571
Training Epoch: 1 [25800/36450]	Loss: 3009.7075
Training Epoch: 1 [25850/36450]	Loss: 3194.6143
Training Epoch: 1 [25900/36450]	Loss: 3158.1770
Training Epoch: 1 [25950/36450]	Loss: 2981.0803
Training Epoch: 1 [26000/36450]	Loss: 3154.5928
Training Epoch: 1 [26050/36450]	Loss: 3021.7097
Training Epoch: 1 [26100/36450]	Loss: 3011.2358
Training Epoch: 1 [26150/36450]	Loss: 3077.8240
Training Epoch: 1 [26200/36450]	Loss: 2976.0491
Training Epoch: 1 [26250/36450]	Loss: 3087.9351
Training Epoch: 1 [26300/36450]	Loss: 3112.7573
Training Epoch: 1 [26350/36450]	Loss: 3031.2236
Training Epoch: 1 [26400/36450]	Loss: 3024.5813
Training Epoch: 1 [26450/36450]	Loss: 3105.3594
Training Epoch: 1 [26500/36450]	Loss: 3128.8323
Training Epoch: 1 [26550/36450]	Loss: 3145.7976
Training Epoch: 1 [26600/36450]	Loss: 3042.5300
Training Epoch: 1 [26650/36450]	Loss: 3071.1973
Training Epoch: 1 [26700/36450]	Loss: 3044.6948
Training Epoch: 1 [26750/36450]	Loss: 29

...

Training Epoch: 17 [32200/36450]	Loss: 512.2725
Training Epoch: 17 [32250/36450]	Loss: 520.6943
Training Epoch: 17 [32300/36450]	Loss: 490.5098
Training Epoch: 17 [32350/36450]	Loss: 524.1516
Training Epoch: 17 [32400/36450]	Loss: 519.2769
Training Epoch: 17 [32450/36450]	Loss: 491.0845
Training Epoch: 17 [32500/36450]	Loss: 577.6346
Training Epoch: 17 [32550/36450]	Loss: 511.4006
Training Epoch: 17 [32600/36450]	Loss: 518.7645
Training Epoch: 17 [32650/36450]	Loss: 561.4418
Training Epoch: 17 [32700/36450]	Loss: 537.8473
Training Epoch: 17 [32750/36450]	Loss: 539.3918
Training Epoch: 17 [32800/36450]	Loss: 517.5081
Training Epoch: 17 [32850/36450]	Loss: 525.1277
Training Epoch: 17 [32900/36450]	Loss: 506.3789
Training Epoch: 17 [32950/36450]	Loss: 493.0305
Training Epoch: 17 [33000/36450]	Loss: 484.2634
Training Epoch: 17 [33050/36450]	Loss: 505.2480
Training Epoch: 17 [33100/36450]	Loss: 483.5373
Training Epoch: 17 [33150/36450]	Loss: 502.7089
Training Epoch: 17 [33200/36450]	Loss: 5

Training Epoch: 18 [4350/36450]	Loss: 514.0752
Training Epoch: 18 [4400/36450]	Loss: 527.7245
Training Epoch: 18 [4450/36450]	Loss: 528.0886
Training Epoch: 18 [4500/36450]	Loss: 506.2036
Training Epoch: 18 [4550/36450]	Loss: 520.4133
Training Epoch: 18 [4600/36450]	Loss: 520.5473
Training Epoch: 18 [4650/36450]	Loss: 501.0227
Training Epoch: 18 [4700/36450]	Loss: 499.5633
Training Epoch: 18 [4750/36450]	Loss: 499.9669
Training Epoch: 18 [4800/36450]	Loss: 505.1364
Training Epoch: 18 [4850/36450]	Loss: 482.6946
Training Epoch: 18 [4900/36450]	Loss: 512.3708
Training Epoch: 18 [4950/36450]	Loss: 494.5989
Training Epoch: 18 [5000/36450]	Loss: 505.8137
Training Epoch: 18 [5050/36450]	Loss: 489.2839
Training Epoch: 18 [5100/36450]	Loss: 541.4110
Training Epoch: 18 [5150/36450]	Loss: 472.4477
Training Epoch: 18 [5200/36450]	Loss: 507.1529
Training Epoch: 18 [5250/36450]	Loss: 494.5620
Training Epoch: 18 [5300/36450]	Loss: 517.1542
Training Epoch: 18 [5350/36450]	Loss: 493.3574
Training Epoc

Training Epoch: 18 [13050/36450]	Loss: 512.1226
Training Epoch: 18 [13100/36450]	Loss: 496.1643
Training Epoch: 18 [13150/36450]	Loss: 504.2034
Training Epoch: 18 [13200/36450]	Loss: 533.8223
Training Epoch: 18 [13250/36450]	Loss: 505.5760
Training Epoch: 18 [13300/36450]	Loss: 494.1448
Training Epoch: 18 [13350/36450]	Loss: 501.3144
Training Epoch: 18 [13400/36450]	Loss: 494.4683
Training Epoch: 18 [13450/36450]	Loss: 519.5241
Training Epoch: 18 [13500/36450]	Loss: 499.2430
Training Epoch: 18 [13550/36450]	Loss: 487.6826
Training Epoch: 18 [13600/36450]	Loss: 519.8177
Training Epoch: 18 [13650/36450]	Loss: 533.3890
Training Epoch: 18 [13700/36450]	Loss: 510.5882
Training Epoch: 18 [13750/36450]	Loss: 521.5488
Training Epoch: 18 [13800/36450]	Loss: 514.0828
Training Epoch: 18 [13850/36450]	Loss: 505.9979
Training Epoch: 18 [13900/36450]	Loss: 517.5496
Training Epoch: 18 [13950/36450]	Loss: 499.6981
Training Epoch: 18 [14000/36450]	Loss: 520.9756
Training Epoch: 18 [14050/36450]	Loss: 4

Training Epoch: 18 [21600/36450]	Loss: 520.6364
Training Epoch: 18 [21650/36450]	Loss: 513.8151
Training Epoch: 18 [21700/36450]	Loss: 491.2439
Training Epoch: 18 [21750/36450]	Loss: 473.3801
Training Epoch: 18 [21800/36450]	Loss: 487.8795
Training Epoch: 18 [21850/36450]	Loss: 520.3171
Training Epoch: 18 [21900/36450]	Loss: 481.8872
Training Epoch: 18 [21950/36450]	Loss: 492.3342
Training Epoch: 18 [22000/36450]	Loss: 502.6955
Training Epoch: 18 [22050/36450]	Loss: 505.7941
Training Epoch: 18 [22100/36450]	Loss: 504.8688
Training Epoch: 18 [22150/36450]	Loss: 490.7375
Training Epoch: 18 [22200/36450]	Loss: 507.8543
Training Epoch: 18 [22250/36450]	Loss: 486.3782
Training Epoch: 18 [22300/36450]	Loss: 475.1465
Training Epoch: 18 [22350/36450]	Loss: 481.7007
Training Epoch: 18 [22400/36450]	Loss: 494.2363
Training Epoch: 18 [22450/36450]	Loss: 490.0075
Training Epoch: 18 [22500/36450]	Loss: 483.9249
Training Epoch: 18 [22550/36450]	Loss: 520.6625
Training Epoch: 18 [22600/36450]	Loss: 4

Training Epoch: 18 [30150/36450]	Loss: 495.0615
Training Epoch: 18 [30200/36450]	Loss: 539.5411
Training Epoch: 18 [30250/36450]	Loss: 481.0726
Training Epoch: 18 [30300/36450]	Loss: 505.5510
Training Epoch: 18 [30350/36450]	Loss: 522.9572
Training Epoch: 18 [30400/36450]	Loss: 487.7372
Training Epoch: 18 [30450/36450]	Loss: 504.6219
